# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 60 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2304.02037


extracting tarball to tmp_2304.02037...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: LatexWarning: Found documentclass in tmp_2304.02037/paper.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'Figures/01caption/figure_ew_vs_mom2' from 'tmp_2304.02037/Figures/01caption/figure_ew_vs_mom2.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'Figures/01caption/figure_spectra_ring_region' from 'tmp_2304.02037/Figures/01caption/figure_spectra_ring_region.tex'
  warnings.warn(LatexWarning(f"Lat

...

Found 132 bibliographic references in tmp_2304.02038/excite_HI.bbl.
Retrieving document from  https://arxiv.org/e-print/2304.02139


extracting tarball to tmp_2304.02139...

 done.
Retrieving document from  https://arxiv.org/e-print/2304.02615


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: LatexWarning: Found documentclass in tmp_2304.02139/rnaas.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2304.02615... done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.02037-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.02037) | **Kinematic analysis of the super-extended HI disk of the nearby spiral  galaxy M83**  |
|| C. Eibensteiner, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>S. Stuber</mark> |
|*Appeared on*| *2023-04-06*|
|*Comments*| *accepted for publication in A&A; 16 pages, 12 figures (+8 pages appendix)*|
|**Abstract**| We present new HI observations of the nearby massive spiral galaxy M83, taken with the VLA at $21^{\prime\prime}$ angular resolution ($\approx500$ pc) of an extended ($\sim$1.5 deg$^2$) 10-point mosaic combined with GBT single dish data. We study the super-extended HI disk of M83 (${\sim}$50 kpc in radius), in particular disc kinematics, rotation and the turbulent nature of the atomic interstellar medium. We define distinct regions in the outer disk ($r_{\rm gal}>$central optical disk), including ring, southern area, and southern and northern arm. We examine HI gas surface density, velocity dispersion and non-circular motions in the outskirts, which we compare to the inner optical disk. We find an increase of velocity dispersion ($\sigma_v$) towards the pronounced HI ring, indicative of more turbulent HI gas. Additionally, we report over a large galactocentric radius range (until $r_{\rm gal}{\sim}$50 kpc) that $\sigma_v$ is slightly larger than thermal (i.e. $>8$km s$^{-1}$ ). We find that a higher star formation rate (as traced by FUV emission) is not always necessarily associated with a higher HI velocity dispersion, suggesting that radial transport could be a dominant driver for the enhanced velocity dispersion. We further find a possible branch that connects the extended HI disk to the dwarf irregular galaxy UGCA365, that deviates from the general direction of the northern arm. Lastly, we compare mass flow rate profiles (based on 2D and 3D tilted ring models) and find evidence for outflowing gas at r$_{\rm gal}$ $\sim$2 kpc, inflowing gas at r$_{\rm gal}$ $\sim$5.5 kpc and outflowing gas at r$_{\rm gal}$ $\sim$14 kpc. We caution that mass flow rates are highly sensitive to the assumed kinematic disk parameters, in particular, to the inclination. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.02038-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.02038) | **Measuring the photo-ionization rate, neutral fraction and mean free path  of HI ionizing photons at $4.9 \leq z \leq 6.0$ from a large sample of  XShooter and ESI spectra**  |
|| <mark>P. Gaikwad</mark>, et al. -- incl., <mark>F. Nasir</mark> |
|*Appeared on*| *2023-04-06*|
|*Comments*| *30 pages (+14 pages appendices), 14 figures (+18 figures appendices); submitted to MNRAS; Main results are summarized in Fig. 10, Fig. 11 and Table 3*|
|**Abstract**| We measure the mean free path ($\lambda_{\rm mfp,HI}$), photo-ionization rate ($\langle \Gamma_{\rm HI} \rangle$) and neutral fraction ($\langle f_{\rm HI} \rangle$) of hydrogen in 12 redshift bins at $4.85<z<6.05$ from a large sample of moderate resolution XShooter and ESI QSO absorption spectra. The fluctuations in ionizing radiation field are modeled by post-processing simulations from the Sherwood suite using our new code ''EXtended reionization based on the Code for Ionization and Temperature Evolution'' (EX-CITE). EX-CITE uses efficient Octree summation for computing intergalactic medium attenuation and can generate large number of high resolution $\Gamma_{\rm HI}$ fluctuation models. Our simulation with EX-CITE shows remarkable agreement with simulations performed with the radiative transfer code Aton and can recover the simulated parameters within $1\sigma$ uncertainty. We measure the three parameters by forward-modeling the Ly$\alpha$ forest and comparing the effective optical depth ($\tau_{\rm eff, HI}$) distribution in simulations and observations. The final uncertainties in our measured parameters account for the uncertainties due to thermal parameters, modeling parameters, observational systematics and cosmic variance. Our best fit parameters show significant evolution with redshift such that $\lambda_{\rm mfp,HI}$ and $\langle f_{\rm HI} \rangle$ decreases and increases by a factor $\sim 6$ and $\sim 10^{4}$, respectively from $z \sim 5$ to $z \sim 6$. By comparing our $\lambda_{\rm mfp,HI}$, $\langle \Gamma_{\rm HI} \rangle$ and $\langle f_{\rm HI} \rangle$ evolution with that in state-of-the-art Aton radiative transfer simulations and the Thesan and CoDa-III simulations, we find that our best fit parameter evolution is consistent with a model in which reionization completes by $z \sim 5.2$. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.02139-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.02139) | **Three long period transiting giant planets from TESS**  |
|| R. Brahm, et al. -- incl., <mark>T. Trifonov</mark>, <mark>J. Eberhardt</mark> |
|*Appeared on*| *2023-04-06*|
|*Comments*| *24 pages, 16 figures, accepted in AJ*|
|**Abstract**| We report the discovery and orbital characterization of three new transiting warm giant planets. These systems were initially identified as presenting single transit events in the light curves generated from the full frame images of the Transiting Exoplanet Survey Satellite (TESS). Follow-up radial velocity measurements and additional light curves were used to determine the orbital periods and confirm the planetary nature of the candidates. The planets orbit slightly metal-rich late F- and early G-type stars. We find that TOI 4406b has a mass of $M_P$= 0.30 $\pm$ 0.04 $M_J$ , a radius of $R_P$= 1.00 $\pm$ 0.02 $R_J$ , and a low eccentricity orbit (e=0.15 $\pm$ 0.05) with a period of P= 30.08364 $\pm$ 0.00005 d . TOI 2338b has a mass of $M_P$= 5.98 $\pm$ 0.20 $M_J$ , a radius of $R_P$= 1.00 $\pm$ 0.01 $R_J$ , and a highly eccentric orbit (e= 0.676 $\pm$ 0.002 ) with a period of P= 22.65398 $\pm$ 0.00002 d . Finally, TOI 2589b has a mass of $M_P$= 3.50 $\pm$ 0.10 $M_J$ , a radius of $R_P$= 1.08 $\pm$ 0.03 $R_J$ , and an eccentric orbit (e = 0.522 $\pm$ 0.006 ) with a period of P= 61.6277 $\pm$ 0.0002 d . TOI 4406b and TOI 2338b are enriched in metals compared to their host stars, while the structure of TOI 2589b is consistent with having similar metal enrichment to its host star. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.02615-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.02615) | **Formation and Destiny of White Dwarf and Be Star Binaries**  |
|| C. Zhu, G. Lü, X. Lu, <mark>J. He</mark> |
|*Appeared on*| *2023-04-06*|
|*Comments*| *17 pages, 12 figures, 2 table, accepted for publication in RAA*|
|**Abstract**| The binary systems consisting of a Be star and a white dwarf (BeWDs) are very interesting.They can originate from the binaries composed of a Be star and a subdwarf O or B star (BesdOBs), and they can merge into red giants via luminous red nova or can evolve into double WD potentially detected by $LISA$ mission. Using the method of population synthesis, we investigate the formation and the destiny of BeWDs,and discuss the effects of the metallicity ($Z$) and the common envelope evolution parameters. We find that BesdOBs are significant progenitors of BeWDs. About 30\% ($Z=0.0001$)-50\% ($Z=0.02$) of BeWDs come from BesdOBs. About 60\% ($Z=0.0001$) -70\% ($Z=0.02$) of BeWDs turn into red giants via a merger between a WD and a non-degenerated star. About 30\% ($Z=0.0001$) -40\% ($Z=0.02$) of BeWDs evolve into double WDs which are potential gravitational waves of $LISA$ mission at a frequency band between about $3\times10^{-3}$ and $3\times10^{-2}$ Hz. The common envelope evolution parameter introduces an uncertainty with a factor of about 1.3 on BeWD populations in our simulations. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2304.02037.md
    + _build/html/tmp_2304.02037/./Figures/plots/gbt_obs_v2.png
    + _build/html/tmp_2304.02037/./Figures/plots/ring_region_spectra.png
    + _build/html/tmp_2304.02037/./Figures/plots/ew_vs_mom2_v2.png
exported in  _build/html/2304.02038.md


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\todo}{\ifmmode \text{\color{red}\Huge{\(\bullet\)}} \else{\color{red}{\Huge\bullet}}\fi}$
$\newcommand{\comment}[2]{{\color{gray} #1:~ #2}}$
$\newcommand$
$\newcommand{  \hi       }{\ifmmode{\rm H} \textsc{i} \else H \textsc{i}\fi}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\vlosmdl}{{V_\mathrm{los, mdl}}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\uv}{{u{-}v }}$
$\newcommand{\ubonn}{Argelander-Institut für Astronomie, Universität Bonn, Auf dem Hügel 71, 53121 Bonn, Germany}$
$\newcommand{\osu}{Department of Astronomy, The Ohio State University, 4055 McPherson Laboratory, 140 West 18th Ave, Columbus, OH 43210, USA}$
$\newcommand{\oan}{Observatorio Astronómico Nacional (IGN), C/ Alfonso XII, 3, E-28014 Madrid, Spain}$
$\newcommand{\inaf}{INAF — Osservatorio Astrofisico di Arcetri, Largo E. Fermi 5, I-50125, Florence, Italy}$
$\newcommand{\mpe}{Max-Planck-Institut für Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}$
$\newcommand{\rechen}{Astronomisches Rechen-Institut, Zentrum f{ü}r Astronomie der Universit{ä}t Heidelberg, M{ö}nchhofstra{\ss}e 12-14, 69120 Heidelberg, Germany}$
$\newcommand{\ox}{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}$
$\newcommand{\zah}{Universität Heidelberg, Zentrum für Astronomie, Institut für theoretische Astrophysik, Albert-Ueberle-Stra{\ss}e 2, 69120, Heidelberg, Germany}$
$\newcommand{\anu}{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}$
$\newcommand{\astrothreed}{ARC Centre of Excellence for All Sky Astrophysics in 3 Dimensions (ASTRO 3D), Australia}$
$\newcommand{\mpia}{Max Planck Institute for Astronomy, Königstuhl 17, D-69117 Heidelberg, Germany}$
$\newcommand{\hsc}{Centro de Desarrollos Tecnológicos, Observatorio de Yebes (IGN), 19141 Yebes, Guadalajara, Spain}$
$\newcommand{\zw}{Universität Heidelberg, Interdisziplinäres Zentrum für Wissenschaftliches Rechnen, Im Neuenheimer Feld 205, 69120 Heidelberg, Germany}$
$\newcommand{\gent}{Sterrenkundig Observatorium, Universiteit Gent, Krijgslaan 281 S9, B-9000 Gent, Belgium}$
$\newcommand{\iram}{Institut de Radioastronomie Millimétrique (IRAM), 300 Rue de la Piscine, F-38406 Saint Martin d’Hères, France}$
$\newcommand{\lerma}{LERMA, Observatoire de Paris, PSL Research University, CNRS, Sorbonne Universités, 75014 Paris, France}$
$\newcommand{\ucsd}{Center for Astrophysics and Space Sciences, Department of Physics, University of California San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}$
$\newcommand{\mmu}{Department of Physics and Astronomy, McMaster University, 1280 Main Street West, Hamilton, ON L8S 4M1, Canada}$
$\newcommand{\cita}{Canadian Institute for Theoretical Astrophysics (CITA), University of Toronto, 60 St George Street, Toronto, ON M5S 3H8, Canada}$
$\newcommand{\wyo}{Department of Physics \& Astronomy, University of Wyoming, Laramie, WY, 82071, USA}$
$\newcommand{\alb}{Dept. of Physics, University of Alberta, Edmonton, Alberta, Canada T6G 2E1}$
$\newcommand{\cape}{Department of Astronomy, University of Cape Town, Private Bag X3, Rondebosch 7701, South Africa}$
$\newcommand{\westv}{Department of Physics and Astronomy, West Virginia University, White Hall, Box 6315, Morgantown, WV 26506, USA}$
$\newcommand{\westvg}{Center for Gravitational Waves and Cosmology, West Virginia University, Chestnut Ridge Research Building, Morgantown, WV 26505, USA}$
$\newcommand{\eso}{European Southern Observatory, Karl-Schwarzschild Straße 2, D-85748 Garching bei M{ü}nchen, Germany}$
$\newcommand{\ulyon}{Univ Lyon, Univ Lyon1, ENS de Lyon, CNRS, Centre de Recherche Astrophysique de Lyon UMR5574, F-69230 Saint-Genis-Laval France}$
$\newcommand{\cfa}{Center for Astrophysics, Harvard \& Smithsonian, 60 Garden St., 02138 Cambridge, MA, USA}$
$\newcommand{\hopk}{Department of Physics \& Astronomy, Bloomberg Center for Physics and Astronomy, Johns Hopkins University, 3400 N. Charles Street, Baltimore, MD 21218}$
$\newcommand{\theoheid}{Institüt  für Theoretische Astrophysik, Zentrum für Astronomie der Universität Heidelberg, Albert-Ueberle-Strasse 2, 69120 Heidelberg, Germany}$
$\newcommand{\madrid}{Departamento de Fisica de la Tierra y Astrofisica \& IPARCOS, Facultad de CC Fisicas, Universidad Complutense de Madrid, 28040, Madrid, Spain}$
$\newcommand{\cool}{Cosmic Origins Of Life (COOL) Research DAO, coolresearch.io}$
$\newcommand{\eff}{Max-Planck-Institut für Radioastronomie, Radioobservatorium Effelsberg, Max-Planck-Strasse 28, Germany}$
$\newcommand{\nrao}{National Radio Astronomy Observatory, 1003 Lopezville Road, Socorro, NM 87801, USA}$
$\newcommand{\nraoc}{National Radio Astronomy Observatory, 520 Edgemont Rd, Charlottesville, VA 22903, USA}$
$\newcommand{\astron}{Netherlands Institute for Radio Astronomy (ASTRON),  Oude Hoogeveensedijk 4, 7991 PD Dwingeloo, Netherlands}$
$\newcommand{\kapeyn}{Kapteyn Astronomical Institute, University of Groningen, PO Box 800, 9700 AV Groningen, The Netherlands}$
$\newcommand{\uct}{Department of Astronomy, University of Cape Town, Private Bag X3, 7701 Rondebosch, South Africa}$
$\newcommand{\liverpool}{Astrophysics Research Institute, Liverpool John Moores University, 146 Brownlow Hill, Liverpool L3 5RF, UK}$
$\newcommand{\equationautorefname}{Eq.}$
$\newcommand{\sectionautorefname}{Section}$
$\newcommand{\subsectionautorefname}{Section}$
$\newcommand{\subsubsectionautorefname}{Section}$
$\newcommand{\arraystretch}{1.0}$</div>



<div id="title">

# Kinematic analysis of the super-extended HI disk of the nearby spiral galaxy M 83$\thanks{Based on observations carried out with the Karl G. Jansky Very Large Array (VLA). The National Radio Astronomy Observatory is a facility of the National Science Foundation operated under cooperative agreement by Associated Universities, Inc.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2304.02037-b31b1b.svg)](https://arxiv.org/abs/2304.02037)<mark>Appeared on: 2023-04-06</mark> -  _accepted for publication in A&A; 16 pages, 12 figures (+8 pages appendix)_

</div>
<div id="authors">

C. Eibensteiner, et al. -- incl., <mark>E. Schinnerer</mark>, <mark>S. Stuber</mark>

</div>
<div id="abstract">

**Abstract:** We present new $\hi$ observations of the nearby massive spiral galaxy $\gal$ taken with the VLA at $21\arcsec$ angular resolution ( $\approx500$ pc) of an extended ( $\sim$ 1.5 deg $^2$ ) 10-point mosaic combined with GBT single dish data. We study the super-extended $\hi$ disk of M83 ( ${\sim}$ 50 kpc in radius), in particular disc kinematics, rotation and the turbulent nature of the atomic interstellar medium. We define distinct regions in the outer disk ( $r_{\rm gal}>$ central optical disk), including ring, southern area, and southern and northern arm. We examine $\hi$ gas surface density, velocity dispersion and non-circular motions in the outskirts, which we compare to the inner optical disk. We find an increase of velocity dispersion ( $\sigma_v$ ) towards the pronounced $\hi$ ring, indicative of more turbulent $\hi$ gas. Additionally, we report over a large galactocentric radius range (until $r_{\rm gal}{\sim}$ 50 kpc) that $\sigma_v$ is slightly larger than thermal (i.e. $>8$ $\kms$ ). We find that a higher star formation rate (as traced by FUV emission) is not always necessarily associated with a higher $\hi$ velocity dispersion, suggesting that radial transport could be a dominant driver for the enhanced velocity dispersion. We further find a possible branch that connects the extended $\hi$ disk to the dwarf irregular galaxy UGCA 365, that deviates from the general direction of the northern arm.  Lastly, we compare mass flow rate profiles (based on 2D and 3D tilted ring models) and find evidence for outflowing gas at r $_{\rm gal}$ $\sim$ 2 kpc, inflowing gas at r $_{\rm gal}$ $\sim$ 5.5 kpc and outflowing gas at r $_{\rm gal}$ $\sim$ 14 kpc. We caution that mass flow rates are highly sensitive to the assumed kinematic disk parameters, in particular, to the inclination.

</div>

<div id="div_fig1">

<img src="tmp_2304.02037/./Figures/plots/gbt_obs_v2.png" alt="Fig1" width="100%"/>

**Figure 1. -** We show here the integrated intensity map of the GBT observation with white contours showing the field of view of the VLA observation. We denote the companion galaxies UGCA 365 and NGC 5264. The $\hi$ emission of UGCA 365 is faint relative to \gal(${\sim}$10 K km s$^{-1}$ compared to ${\sim}$400 K km s$^{-1}$).  (*fig:gbt*)

</div>
<div id="div_fig2">

<img src="tmp_2304.02037/./Figures/plots/ring_region_spectra.png" alt="Fig2" width="100%"/>

**Figure 2. -** Examples of four spectra of individual line of sights that show higher values in the velocity map. These show the multi-component behavior of the spectra. (*app:spectra_ring_region*)

</div>
<div id="div_fig3">

<img src="tmp_2304.02037/./Figures/plots/ew_vs_mom2_v2.png" alt="Fig3" width="100%"/>

**Figure 3. -**  Comparison of two approaches to determine the velocity dispersion. In all panels, the y-axes show the \sigmom  and the x-axes the \sigeff values colored by the regions defined within this work (see the mask in \autoref{fig:diff_reg}). Only the upper left panel shows the x and y axes in linear scale, all the others are in log scale. We show linear regression fits between \sigmom  and \sigeff for the central disk (yellow), ring (orange), southern area (red), southern arm (purple), and northern arm (black). We see the highest scatter in \sigmom in the ring region of ${\sim}2.5$dex. This is likely to represent a multi-component spectrum. (*fig:ew_vs_mom2*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2304.02037"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\PG}[1]{{\color{red} \bf #1}}$
$\newcommand{\PGADD}[1]{{   #1}}$
$\newcommand{\TRC}[1]{{\color{green} \bf[TRC:  #1]}}$
$\newcommand{\RS}[1]{{\color{blue}\bf[RS:  #1]}}$
$\newcommand{\mgh}[1]{{\color{magenta}\bf[MGH:  #1]}}$
$\newcommand{\WHITE}[1]{{\color{white} \bf   #1}}$
$\newcommand{\RED}[1]{{\color{red} \bf   #1}}$
$\newcommand{\GREEN}[1]{{\color{OliveGreen} \bf   #1}}$
$\newcommand{\BLUE}[1]{{\color{blue} \bf   #1}}$
$\newcommand{\MAGENTA}[1]{{\color{magenta} \bf   #1}}$</div>



<div id="title">

# Measuring the photo-ionization rate, neutral fraction and mean free path of HI ionizingphotons at $\zrange{4.9}{6.0}$ from a large sample of XShooter and ESI spectra $\vspace{-6mm}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2304.02038-b31b1b.svg)](https://arxiv.org/abs/2304.02038)<mark>Appeared on: 2023-04-06</mark> -  _30 pages (+14 pages appendices), 14 figures (+18 figures appendices); submitted to MNRAS; Main results are summarized in Fig. 10, Fig. 11 and Table 3_

</div>
<div id="authors">

<mark>P. Gaikwad</mark>, et al. -- incl., <mark>F. Nasir</mark>

</div>
<div id="abstract">

**Abstract:** We measure the mean free path ( $\lambda_{\rm mfp,HI}$ ), photo-ionization rate( $\langle \Gamma_{\rm HI} \rangle$ ) and neutral fraction ( $\langle f_{\rm HI}\rangle$ ) of hydrogen in 12 redshift bins at $4.85<z<6.05$ from a large sampleof moderate resolution XShooter and ESI QSO absorption spectra. Thefluctuations in ionizing radiation field are modeled by post-processingsimulations from the $\sherwood$ suite using our new code"EXtended reionization based on the Code for Ionization and Temperature Evolution"( ${\sc ex-cite}$ ). ${\sc ex-cite}$ uses efficient Octree summation for computingintergalactic medium attenuation and can generate large number of highresolution $\GHI$ fluctuation models. Our simulation with ${\sc ex-cite}$ showsremarkable agreement with simulations performed with the radiative transfercode $\aton$ and can recover the simulated parameters within $1\sigma$ uncertainty.  We measure the three parameters by forward-modeling the $\lya$ forest and comparing the effective optical depth ( $\tau_{\rm eff, HI}$ ) distribution insimulations and observations. The final uncertainties in our measuredparameters account for the uncertainties due to thermal parameters, modelingparameters, observational systematics and cosmic variance. Our best fitparameters show significant evolution with redshift such that $\lambda_{\rmmfp,HI}$ and $\langle f_{\rm HI} \rangle$ decreases and increases by a factor $\sim 6$ and $\sim 10^{4}$ , respectively from $z \sim 5$ to $z \sim 6$ . Bycomparing our $\lambda_{\rm mfp,HI}$ , $\langle \Gamma_{\rm HI} \rangle$ and $\langle f_{\rm HI} \rangle$ evolution with that in state-of-the-art $\aton$ radiative transfer simulations and the $\thesan$ and $\codathree$ simulations, wefind that our best fit parameter evolution is consistent with a model in whichreionization completes by $z \sim 5.2$ .  In our best fit model, the neutralislands  persist down to $z \sim 5.2$ giving rise to the long dark troughs seenin the observed spectra. Our best fit model that matches the $\tau_{\rm eff, HI}$ distribution also reproduces the dark gap length distribution and transmissionspike height distribution suggesting  robustness and accuracy of our measuredparameters.

</div>

<div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2304.02038"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

185  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
